# Project: planning my next holidays ☀️

Let's create a script that allows to get some information about all the hotels in a given city on <a href="https://www.booking.com" target="_blank">www.booking.com</a> 🧙

**We strongly recommend that you use Scrapy, it will be much easier!**

You can scrap as many information as you want, but we suggest that you get at least:

* The hotel name, 
* The url to its booking.com page, 
* Its coordinates: latitude and longitude,
* The score given by the website users,
* The text description of the hotel.

Then, you can execute this script for several cities from yesterday's list. Make sure you save the results in different files for each city and that the name of the city is stored in the filename (for later purposes 😉).

In [93]:
import pandas as pd 
from sqlalchemy import create_engine
import plotly.express as px
import logging
import os 

In [7]:
engine = create_engine("postgresql+psycopg2://", echo=True)

In [36]:
path_to_file="/Users/saas1/OneDrive/Escritorio/introduction_to_ python_for_data_science/Projects/Project_3/Project/"

data_frame_1=pd.read_csv (path_to_file +"/cities.csv",index_col='Unnamed: 0')
data_frame_2=pd.read_excel(path_to_file +"/hotel.xlsx",index_col='Unnamed: 0')

data_frame_2.shape


(12148, 7)

In [37]:
data_frame_2= data_frame_2.drop(['url', 'description'], axis=1)
data_frame_2.dropna(subset=['score'], inplace=True)
data_frame_2['score'] = data_frame_2['score'].str.replace(',','.')
data_frame_2['score'] = data_frame_2['score'].astype(float)
data_frame_2.head()

,name,score,city,lon,lat
0,Hôtel Vert,8.1,Mont Saint Michel,-1.509617,48.614700
1,Mercure Mont Saint Michel,8.2,Mont Saint Michel,-1.510545,48.614247
2,Hotel De La Digue,7.1,Mont Saint Michel,-1.510918,48.616882
3,Le Saint Aubert,7.3,Mont Saint Michel,-1.510105,48.612938
4,Les Terrasses Poulard,7.3,Mont Saint Michel,-1.510379,48.635349


In [15]:
data_frame_1.to_sql("cities",engine)


In [38]:
best_5=["Collioure", "Marseille", "Cassis","Bayonne", "Bormes les Mimosas"]

data_frame_2=data_frame_2[data_frame_2['city'].isin(best_5)]


In [39]:
data_frame_2.to_sql("hotels_france",engine)

2021-10-18 19:48:39,513 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2021-10-18 19:48:39,514 INFO sqlalchemy.engine.base.Engine {'name': 'hotels_france'}
2021-10-18 19:48:39,889 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE hotels_france (
	index BIGINT, 
	name TEXT, 
	score FLOAT(53), 
	city TEXT, 
	lon FLOAT(53), 
	lat FLOAT(53)
)


2021-10-18 19:48:39,891 INFO sqlalchemy.engine.base.Engine {}
2021-10-18 19:48:40,155 INFO sqlalchemy.engine.base.Engine COMMIT
2021-10-18 19:48:40,280 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_hotels_france_index ON hotels_france (index)
2021-10-18 19:48:40,281 INFO sqlalchemy.engine.base.Engine {}
2021-10-18 19:48:40,530 INFO sqlalchemy.engine.base.Engine COMMIT
2021-10-18 19:48:40,656 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-10-18 19:48:40,675 INFO sqlalchemy.engine.base.Engine INSERT INTO hotels_

In [50]:
from sqlalchemy.sql import text

stmt = text("SELECT * FROM hotels_france ")

df_hotels = pd.read_sql(
        stmt,
        engine
    )



from sqlalchemy.sql import text

stmt = text("SELECT * FROM cities ")

df_cities = pd.read_sql(
        stmt,
        engine
    )


2021-10-18 20:04:25,195 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2021-10-18 20:04:25,198 INFO sqlalchemy.engine.base.Engine {'name': 'SELECT * FROM hotels_france '}
2021-10-18 20:04:25,584 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM hotels_france 
2021-10-18 20:04:25,585 INFO sqlalchemy.engine.base.OptionEngine {}
2021-10-18 20:04:26,367 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2021-10-18 20:04:26,368 INFO sqlalchemy.engine.base.Engine {'name': 'SELECT * FROM cities '}
2021-10-18 20:04:26,770 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM cities 
2021-10-18 20:04:26,771 INFO sqlalchemy.engine.base.OptionEngine {}


In [91]:
df_hotels=df_hotels.sort_values(by='score', ascending=False).reset_index(drop=True)

best_5=["Collioure", "Marseille", "Cassis","Bayonne", "Bormes les Mimosas"]
hotels_df=pd.DataFrame()

for i in best_5:
    hotels_df=hotels_df.append(df_hotels[df_hotels["city"]==i].iloc[0:20,:])

hotels_df.reset_index(drop=True)

,index,name,score,city,lon,lat
0,9543,Boramar,9.7,Collioure,3.085355,42.527543
1,9547,Michelet,9.7,Collioure,3.081826,42.526520
2,9611,Le Mouret,9.5,Collioure,3.084889,42.528223
3,9579,Studio Canta la Mar - Vue exceptionnelle,9.5,Collioure,3.090316,42.524431
4,9661,Le 15-Appartments Collioure,9.5,Collioure,3.078625,42.528518
...,...,...,...,...,...,...
95,6950,Les Bastides du Haut Para,9.0,Bormes les Mimosas,6.325990,43.145951
96,6978,Apartment Le Jardin du Loderi-1,9.0,Bormes les Mimosas,6.343075,43.144703
97,6906,Le petit atelier,9.0,Bormes les Mimosas,6.349179,43.142501
98,6946,Les ammonites,9.0,Bormes les Mimosas,6.356524,43.122856


In [109]:
fig = px.scatter_mapbox(hotels_df, title="Best Hotels",lat="lat", lon="lon", color="score", zoom =5.5,size="score",size_max=10,
                        mapbox_style="open-street-map", color_continuous_scale = 'Bluered', range_color = [8,10.0],hover_name="name"
                       )
fig.show()


In [101]:
df_cities=df_cities.iloc[0:5,]

In [110]:
fig = px.scatter_mapbox(df_cities, lat="lat", lon="lon", color="T_Average", zoom =5.5,size="T_Average",size_max=10,
                        mapbox_style="open-street-map", color_continuous_scale = 'OrRd', range_color = [8,20.0],
                       title="Best Cities",hover_name="city")
fig.show()
